In [7]:
!pip install deepface opencv-python pytube --quiet


from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
from PIL import Image
import io

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
  ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename


filename = take_photo()
print("Photo captured and saved as:", filename)


from deepface import DeepFace
result = DeepFace.analyze(img_path=filename, actions=['emotion'])
print("Detected Emotion:", result[0]['dominant_emotion'])


import webbrowser


emotion_playlists = {
    'happy': 'https://www.youtube.com/watch?v=ZbZSe6N_BXs&list=PLF797E961509B4EB5',
    'sad': 'https://www.youtube.com/watch?v=ho9rZjlsyYY&list=PL5D7ECFAAC3D19638',
    'angry': 'https://www.youtube.com/watch?v=d-diB65scQU&list=PL4C06BBEA15D39D1D',
    'neutral': 'https://www.youtube.com/watch?v=3JWTaaS7LdU&list=PL8F6B0753B2CCA128',
    'fear' : 'https://www.youtube.com/watch?v=xo1VInw-SKc&list=PLpxWKmiR9bcN6PZ6clo7QcU42f0hT0EmV',
    'surprise': 'https://www.youtube.com/watch?v=Eo-KmOd3i7s',
    'disgust': 'https://www.youtube.com/watch?v=lDK9QqIzhwk'

}

emotion = result[0]['dominant_emotion']
playlist_url = emotion_playlists.get(emotion, None)

if playlist_url:
    print(f"Opening {emotion} playlist:", playlist_url)
    webbrowser.open(playlist_url)
else:
    print("No playlist found for the detected emotion.")




<IPython.core.display.Javascript object>

Photo captured and saved as: photo.jpg
Detected Emotion: happy
Opening happy playlist: https://www.youtube.com/watch?v=ZbZSe6N_BXs&list=PLF797E961509B4EB5
